# Getting Started on the CISL Cloud

```{warning}
This project is still in a pre-production stage and user data stored on our services is not guaranteed. Please utilize code repositories such as GitHub and production storage systems such as GLADE, Stratus, or a public cloud option. This ensures your work is backed up while making it easier to reproduce it quickly in different environments. 
```

The CISL Cloud is built on a container management platform known as Kubernetes(K8s). RKE2 is used to deploy K8s on servers running AlmaLinux. Microservices are utilized to provide a tool set that streamlines common tasks. Examples of common tasks that are eliminated due to these microservices are creating DNS records, assigning TLS certificates, configuring a reverse proxy, load balancing applications, scaling resources, mounting shared volumes, and providing a self healing environment that is highly available.    

## Documentation

This documentation site is meant to include all information you may need to navigate the CISL cloud and the resources hosted on it. If you find anything that is not clear or missing please submit an issue to our GitHub repository via the open issue link that appears when you hover over the GitHub icon at the top of the page. The Report Issue link in the navbar opens a Jira ticket that also can be used to communicate a request for enhancing content.    

This Getting Started guide will provide a high level introduction in to each resource. More in depth details on each resource can be found on this Documentation page.  

## JupyterHub

<span class="d-flex justify-content-center py-4">
    <a href="https://jupyter.k8s.ucar.edu/">
        <figure>
            <img src="https://ncar.github.io/cisl-cloud/_static/jhub-logo.png">
            <figcaption style="text-align:center">K8s JupyterHub</figcaption>
            <figcaption style="text-align:center"><a href="how-to/k8sJH/k8sJH-intro.html">Link to Docs</a></figcaption>
        </figure>
    </a>
</span>

JupyterHub is utilized to provide a Virtual Development Environment. 

### Authentication

Authentication to our JupyterHub is currently controlled via GitHub Teams. Information on how to join the GitHub team can be found at this [how-to request access link](../how-to/k8sJH/access).

### Notebook/Server Options

The JupyterHub uses custom notebook images controlled and maintained by the Cloud team. After logging in you will be redirected a page titled `Server Options`. This is the startup page where you get to chose what resources you want to utilize. We provide a CPU image and two GPU images that can be selected after logging in. All the images provide the same extensions and are setup to provide a cohesive experience while switching between images. The users home directory, `/home/jovyan` for all users, is persistent and retains anything written to it. 

#### Custom Conda Kernels

The custom images all use a python module called `nb_conda_kernels` that allows any conda environment containing `ipykernel` to be registered as a Jupyter kernel automatically. 

#### CPU Image

The CPU image is named `NSF NCAR CPU Notebook` and has 3 different resource options for Memory and CPU that can be selected from the dropdown.

By default the CPU image contains 3 prebuilt conda environments that contain a lot of packages typically required to do scientific analysis. These environments are named:

- cisl-cloud-base
- npl-2023b
- r-4.3

More details on what are included in these packages can be found at this [link to the base conda environments documentation](../how-to/k8sJH/base-conda).

#### GPU Images

Two different GPU images are offered. One is built around the [Tensorflow](https://www.tensorflow.org/) Python package, the other is built around [PyTorch](https://pytorch.org/). The GPU images have all the same custom configurations as the CPU image. They contain only 1 custom conda environment by default named:

- cisl-gpu-base

It contains a lot of the same base packages from the `cisl-cloud-base` environment, but adds in the required GPU packages. 

#### Git Extension

The best place to start is to get some code in to the notebook in order to run through real life scenarios. Our custom image includes a Git extension that can be used to interact with Git repositories via a UI instead of using only the command line. 

#### Dask Gateway

As of right now Dask Gateway is broken. It spins up a cluster, but the distributed Client can't connect to the cluster and times out. We are still investigating this and will update this Cell when it changes. 

## Data Access & File Sharing

### GLADE

Read Only access is provided to GLADE campaign and collections. This is mounted inside all Jupyter environments launched on our JupyterHub on the path `/glade/campaign` & `/glade/collections`. Read Only access to these mounts can also be provided to containerized applications like Web Visualizations and is described in more detail in that section below. 

### Stratus

<span class="d-flex justify-content-center py-4">
    <a href="https://stratus-admin.ucar.edu:10443/asview">
        <figure>
            <img src="https://ncar.github.io/cisl-cloud/_static/stratus.png">
            <figcaption style="text-align:center">Stratus Object Storage</figcaption>
            <figcaption style="text-align:center"><a href="how-to/Stratus/stratus-intro.html">Link to Docs</a></figcaption>
        </figure>
     </a>
</span>

CISL has an object storage platform known as Stratus that can be used to store and read objects. Getting an Access ID and a Secret Key can be done by following this [link to the instructions](../how-to/Stratus/stratus-access). Buckets can also be configured to allow anonymous users read access so data can be read without keys. There is a S3 Object Storage Browser extension installed in the JupyterHub instance. Directions on how to utilize that extension can be found at this [link to the Jupyter S3 Extension documentation](../how-to/Stratus/jupyter-extension). The Object Browser really only lets you browse the contents of buckets tied to the Access ID provided. In order to upload and download objects it's better to use a python module such as `boto3` or `isd_s3`. If you'd like help getting started with either module a good starting place can be found at this [link to S3 Python Examples](../how-to/Stratus/stratus-cmd).

### NFS Share

A shared NFS volume is also mounted in the JupyterHub environment and is found at `/home/jovyan/shared`. As of right now it is only read-only and nothing is being stored on it. This is expected to change as more users come on board and we get ideas on what use cases this would satisfy. Some examples of current ideas for this space would be sharing scripts, notebooks, and other files that would be useful to the community as a whole.  

## Web Visualizations

```{note}
Some of the topics discussed here may be outside of your realm of expertise. The Cloud team is here to help you through all of these steps. If you have an application you want to host on the cloud but have no idea on how to get started please follow this [link to instructions on creating a ticket for the team to assist you](../how-to/create-tickets).
```

Hosting Interactive Web Visualizations, and any other containerized application, is made easy with Kubernetes. There are a number of different frameworks when it comes to running a web application. Whether it's a Python based application like Flask, Django, FastAPI, Bokeh, Panel, or HTML with JavaScript, running a containerized version of your application opens up the ability to run it easily in a variety of places. The following guide assumes that you are already familiar with creating applications with some of the web frameworks listed above.

### Creating a Container Image

If you are able to run your application locally and access via a web browser, via `http://localhost:5001` or `http://127.0.0.1:5001` for instance, it shouldn't be too hard to create a container image for your application. Regardless of whether you use Docker, Podman, or something else, the first place to start when containerizing an application is to create a set of instructions on how to build your application in a file known as a Dockerfile. The instructions are going to vary based on the requirements of the application being built so it's difficult to provide an example that covers everything required for your specific application. There are a set of examples provided at this [link on containerizing a python app](../how-to/K8s/Hosting/containerize). It will walk you through creating a basic Dockerfile, building that image locally, and finally testing it by running it locally as well. If you need any help containerizing a specific application please reach out to the team at the following [email link](mailto:cisl-on-prem-cloud@ucar.edu).

### Hosting your Container on K8s

Now that you have a working container image of your application that you've tested locally and confirmed it works it's ready to be hosted on our K8s platform. This is accomplished by creating a collection of yaml files known as Helm charts to deploy and manage our K8s applications. We have a few different Helm templates that can be used to create your own Helm chart for your applications and requirements. As of right now we have the following templates in GitHub: 

- [Standalone Web Application](https://github.com/NicholasCote/web-app-helm)
- [Web Application with a dedicated Dask Cluster](https://github.com/NicholasCote/web-app-dask-helm)
- [Web Application with a GLADE mount](https://github.com/NicholasCote/web-app-vols-helm)

In all those examples the only files you need to concern yourself with to start are the `values.yaml` & `Chart.yaml` files. `Chart.yaml` mostly just tracks the version of your application and provides a little description and details of what it actually is. The `values.yaml` file is where you provide the image to use, resources required, port to connect to, and names to use. The K8s cluster has services configured to assign your application a hostname, create a resolvable FQDN to browse to, and a TLS certificate to secure the application via HTTPS.  

```{note}
As of right now we only can host applications on the internal UCAR network on the `*.k8s.ucar.edu` domain. We are actively working on public access and will provide an update when this functionality is live. Switching your application from internal to external network access is like flipping a switch so don't wait for external access to make sure your application works properly.
```

### Utilizing CI/CD

<span class="d-flex justify-content-center py-4">
    <a href="how-to/K8s/Hosting/deploy">
        <figure>
            <img src="https://ncar.github.io/cisl-cloud/_static/argo.png">
            <figcaption style="text-align:center">Argo CD Docs</figcaption>
        </figure>
    </a>
</span>

Once the initial Dockerfile and Helm chart has been created it's possible to setup a full CI/CD workflow to deploy your application when new changes are pushed to GitHub. A tool called Argo CD is utilized to automatically sync any changes made to the Helm chart in your GitHub repository to K8s and more information on Argo CD can be found at this [link to the Argo CD documentation](../how-to/K8s/Hosting/deploy). The Cloud team will have to be engaged to help you setup the Argo CD piece initially. Follow this [link to instructions on creating a ticket](../how-to/create-tickets) for the team to assist you with Argo CD. We will also make sure your application deploys correctly as part of the initial deployment. GitHub actions is then utilized to run jobs that will create a new container image for your application, push that to a container registry, and update the `values.yaml` & `Chart.yaml` files. Now when changes are pushed to your GitHub repository a GitHub workflow is kicked off that will update your application image, update the Helm chart, Argo CD will see the change to the Helm chart, and then it will sync it automatically. Your application is now utilizing a full CI/CD workflow to deploy changes. You can also set this up to deploy a development version of your application to test changes before merging the changes to the main branch and updating the production version of your application. 